<a href="https://colab.research.google.com/github/thejarlid/makemore/blob/main/makemore_pt1/Makemore_Pt_1_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt

--2025-07-10 03:04:47--  https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt.2’

names.txt.2         100%[===================>] 222.80K  --.-KB/s    in 0.02s   

2025-07-10 03:04:47 (9.06 MB/s) - ‘names.txt.2’ saved [228145/228145]



In [ ]:
!ls

names.txt  names.txt.1	names.txt.2  sample_data


In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
# now we are going to create a 2d array the rows will be the first character in the bigram and the columns will be the second character in the bigram and then the
# value at the index will be the frequency of that combination

import torch

In [ ]:
N = torch.ones((27, 27, 27), dtype=torch.int32)
N[0, 0, 0] = 0

In [ ]:
# for fast indexing into the 2d array
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
for w in words[:]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2, ix3] += 1

In [ ]:
P = N / N.sum(2, keepdim=True)

In [ ]:
log_liklihood = 0.0
n = 0
for w in words[:]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1, ix2, ix3]
    logprob = torch.log(prob)
    log_liklihood += logprob
    n += 1
print(f'{log_liklihood=}')
negative_log_liklihood = -log_liklihood
print(f'{negative_log_liklihood=}')
print(f'{negative_log_liklihood/n}')

log_liklihood=tensor(-410414.9688)
negative_log_liklihood=tensor(410414.9688)
2.092747449874878


In [ ]:
# Sampling
names = []
for i in range(10):
    out = []
    ix1, ix2 = 0, 0
    while True:
        p = P[ix1, ix2]
        ix1 = ix2
        ix2 = torch.multinomial(p, 1, replacement=True).item()
        if ix2 == 0:
            break
        out.append(itos[ix2])

    names.append("".join(out))

print(names)

['aidyne', 'paikorthelyania', 'son', 'maydiahniyahn', 'siyanield', 'janiemoix', 'oyaster', 'hayn', 'xylyssellyn', 'chaillia']


In [ ]:
# PART 2 -- Neural Network Appraoch

In [ ]:
# GOAL: maximize likelihood of the data wrt model params (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)

In [ ]:
# -------------------- !!! OPTIMIZATION !!! -----------------------

In [ ]:
# create a training set of all the bigrams
xs, ys = [], []

for w in words[:]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1, ix2))
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print('number of examples:', num)

# initialize the network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2,27), generator=g, requires_grad=True)

number of examples: 392226


In [ ]:
import torch.nn.functional as F

# gradient descent
for k in range(200):
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network one-hot encoding
  xenc = xenc.view(-1, 27*2)
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # coutns, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabiliites for next character
  # the last two lines here are together called a 'softmax'
  loss = -probs[torch.arange(len(ys)), ys].log().mean() + 0.05*(W**2).mean() # adding regularization to the end of the loss as well which incentivises non 0 weights
  if k % 10 == 0:
    print(loss.item())

  # backward pass
  W.grad = None # set to zero gradient
  loss.backward()
  # update
  W.data += -50 * W.grad

2.2905490398406982
2.290060520172119
2.2896833419799805
2.289388418197632
2.2891547679901123
2.2889676094055176
2.288816452026367
2.28869366645813
2.288593053817749
2.288510799407959
2.288442611694336
2.2883858680725098
2.2883386611938477
2.288299322128296
2.288266181945801
2.288238525390625
2.288215160369873
2.2881956100463867
2.2881786823272705
2.2881643772125244


In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    # --------
    # BEFORE:
    # p = P[ix]
    # --------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
    xenc = xenc.view(-1, 27*2)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next chatacter
    # --------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

uuoidediahaaazouuufayoairaavuiirltohooaoauezvudaanaauyamilevhaisdbdailrwimoazsnoyieaylartezuavuumehyfoetumjrpoyasuwahouaeiyhoraayaezocaaaeaaadiwerimoiimwynilsaiaasnhavoiuouszohddgosfmataaoiluirruauamuhdtorkrrmruuiaaeuyeiumjeauoemahuuawyhapvhvcararrryduhuadoitauualyiabalahuuiueaumpoabzzhemimayawaahubaodaalhavslrtikizsaloevevonunoewafabuzaemruaatearyhwamurzaodridrdzuohvaliipvrghaeuezrwauliaevywhqeovaeiosvhibdhaueudzmimwaoasaozoiiuvnoaaiaieuewulogyewfdairrlarvmyakmaouanmjrdaaaomaavadrbreuihazvzsrdabavauolwbahrtuattiiabuafdtouaehjaoalyudvudqlasdoahltaumloadmaldfaamravumfslamrdvdihkvogaouvrdsyvivoouauoieuaogodvnvaguuhmruvrafrlaglyuialuohmdoiurryliirsoirevehaiparkazvzyaaeriadauulnlrcicvataaryaeaasobrlrusoahuebazaouejbiouzsnvrarhhhisivaublaoamoivabuaauuzaeazyawhoervalhaeadayrauabalqaoumeyzozbzozrubravluadovvheasulemrconakvnyinzasvwohamyohaaiaemruelwwlsxieeicdloekaahljagkzieiuiaueaionayaulauovyyi.
dlhhhmhyuseuaoziezhmsuaeoyzeadiauouozoamiqhaamradsuuvreioahviyrvtaxhuloltilmolmuoaliaohoheaohauah